# **VWAP-Based Intraday Trading Bot (OANDA API)**

This project implements a fully automated intraday trading system using
real-time market data from OANDA. The strategy is based on a VWAP pullback
continuation model combined with EMA trend confirmation and strict
risk management.

The system:
- Fetches bid/ask-aware OHLC data
- Computes VWAP and EMA indicators
- Generates long/short trade signals
- Executes trades automatically via the OANDA v20 API
- Logs trade entries and exits with realized PnL

⚠️ This project is for educational and research purposes only.
</br>
</br>


#### *Imports & Dependencies*

This section imports:
- OANDA v20 API endpoints for trading and account data
- pandas and pandas-ta for time series analysis
- A local `config.py` file that securely stores API credentials

In [1]:
# OANDA API & endpoint clients
import config
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as transactions

# Data analysis libraries
import pandas as pd
import pandas_ta as ta

#### *OANDA API Connection*

Creates an authenticated client using the OANDA v20 REST API.
All trading, pricing, and account requests are made through this client.

In [2]:
# Initialize authenticated OANDA client
client = API(access_token=config.OANDA_API_KEY)

#### *Strategy Configuration*

Defines the trading instrument, candle timeframe, and historical window
used for indicator calculations.


In [3]:
# Trading configuration
timeframe = "M5"          # 5-minute candles
count = 500               # Number of candles fetched
instrument = "SPX500_USD" # S&P 500 CFD

#### *Market Data Retrieval*

Fetches completed bid/ask candles from OANDA and converts them into a
pandas DataFrame indexed by timestamp.


In [4]:
def get_candles(instrument, timeframe, count, client):
    params = {
        "granularity": timeframe,
        "price": "BA",   # Bid + Ask prices
        "count": count
    }

    r = instruments.InstrumentsCandles(
        instrument=instrument,
        params=params
    )

    client.request(r)
    candles = r.response["candles"]

    data = []

    for c in candles:
        if c["complete"]:        # Skips incomplete candles
            data.append({
                "time": c["time"],
                "ask_open": float(c["ask"]["o"]),
                "ask_high": float(c["ask"]["h"]),
                "ask_low": float(c["ask"]["l"]),
                "ask_close": float(c["ask"]["c"]),
                "bid_open": float(c["bid"]["o"]),
                "bid_high": float(c["bid"]["h"]),
                "bid_low": float(c["bid"]["l"]),
                "bid_close": float(c["bid"]["c"])
            })

    if not data:
        raise ValueError("No completed candles returned")

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"], utc=True)
    df.set_index("time", inplace=True)

    return df


#### *Technical Indicator Calculation*

Computes:
- Mid prices from bid/ask
- VWAP (reset daily)
- 20-period EMA
- Session high and low

NaN values are dropped once indicators stabilize.


In [5]:
def calculate_indicators(df):
    df = df.copy()

    # Mid prices
    df["mid_close"] = (df["ask_close"] + df["bid_close"]) / 2
    df["mid_high"] = (df["ask_high"] + df["bid_high"]) / 2
    df["mid_low"] = (df["ask_low"] + df["bid_low"]) / 2

    # VWAP
    typical_price = (df["mid_high"] + df["mid_low"] + df["mid_close"]) / 3

    df["vwap"] = (
        typical_price.groupby(df.index.date)
        .apply(lambda x: x.cumsum() / (range(1, len(x) + 1)))
        .reset_index(level=0, drop=True)
    )

    # EMA
    df["ema_20"] = ta.ema(df["mid_close"], length=20)

    # Session high / low
    df["session_high"] = df.groupby(df.index.date)["mid_high"].cummax()
    df["session_low"] = df.groupby(df.index.date)["mid_low"].cummin()

    # Clean NaNs
    df = df.dropna(subset=["vwap", "ema_20"])

    return df


#### *Position State Validation*

Prevents duplicate trades by checking whether an open position already
exists for the selected instrument.


In [6]:
def has_open_position(instrument, client):
    r = positions.OpenPositions(config.OANDA_ACCOUNT_ID)
    client.request(r)

    for pos in r.response.get("positions", []):
        if pos["instrument"] == instrument:
            long_units = int(pos["long"]["units"])
            short_units = int(pos["short"]["units"])

            if long_units != 0 or short_units != 0:
                return True

    return False


#### *Trade Logging Structure*

Initializes a DataFrame to track:
- Trade entries and exits
- Risk parameters
- Exit reasons
- Realized PnL

This allows for post-trade analysis and performance evaluation.


In [7]:
trade_log = pd.DataFrame(columns=[
    "timestamp",
    "instrument",
    "direction",
    "entry_price",
    "stop_loss",
    "take_profit",
    "exit_price",
    "exit_reason",
    "pnl"
])

#### *Order Execution*

Places market orders with attached stop loss and take profit levels.
Returns metadata required for trade tracking.


In [8]:
def place_order(direction, stop_loss, take_profit):
    units = 1 if direction == "long" else -1

    data = {
        "order": {
            "instrument": instrument,
            "units": str(units),
            "type": "MARKET",
            "timeInForce": "FOK",
            "stopLossOnFill": {"price": f"{stop_loss:.3f}"},
            "takeProfitOnFill": {"price": f"{take_profit:.3f}"}
        }
    }

    r = orders.OrderCreate(
        config.OANDA_ACCOUNT_ID,
        data=data
    )

    response = client.request(r)

    trade_id = response["orderFillTransaction"]["tradeOpened"]["tradeID"]
    entry_price = float(response["orderFillTransaction"]["price"])
    entry_time = response["orderFillTransaction"]["time"]

    return {
        "trade_id": trade_id,
        "entry_price": entry_price,
        "entry_time": entry_time
    }

#### *VWAP Pullback Continuation Strategy*

Strategy logic:
- Trade only in the direction of trend (EMA-20)
- Enter on pullback through EMA toward VWAP
- Require confirmation candle close
- Use recent swing highs/lows for risk placement
- Enforce minimum positive risk


In [9]:
def vwap_pullback_trade(df):
    global trade_log

    tp_ratio = 2.0  # Risk : Reward

    last_candle = df.iloc[-1]
    previous_candle = df.iloc[-2]
    trade_time = last_candle.name  # index is datetime

    # ----------------------------
    # LONG SETUP
    # ----------------------------
    long_signal = (
        last_candle["mid_close"] > last_candle["vwap"] and
        last_candle["mid_close"] > last_candle["ema_20"] and
        previous_candle["mid_close"] < previous_candle["ema_20"]
    )

    # ----------------------------
    # SHORT SETUP
    # ----------------------------
    short_signal = (
        last_candle["mid_close"] < last_candle["vwap"] and
        last_candle["mid_close"] < last_candle["ema_20"] and
        previous_candle["mid_close"] > previous_candle["ema_20"]
    )

    # ----------------------------
    # POSITION CHECK
    # ----------------------------
    if long_signal or short_signal:
        if has_open_position(instrument, client):
            print("Position already open — skipping trade")
            return

    # ----------------------------
    # LONG TRADE
    # ----------------------------
    if long_signal:
        print("Buy Signal: VWAP pullback continuation")

        entry_price = last_candle["ask_close"]
        stop_loss = df["bid_low"].iloc[-3:-1].min()

        risk = entry_price - stop_loss
        if risk <= 0:
            print("Invalid risk — skipping trade")
            return

        take_profit = entry_price + tp_ratio * risk
        reward = take_profit - entry_price

        place_order(
            direction="long",
            stop_loss=stop_loss,
            take_profit=take_profit
        )

        # ---- LOG TRADE ----
        trade_log.loc[len(trade_log)] = {
            "timestamp": last_candle.name,
            "instrument": instrument,
            "direction": "long",
            "entry_price": entry_price,
            "stop_loss": stop_loss,
            "take_profit": take_profit,
            "exit_price": None,
            "exit_reason": None,
            "pnl": None
        }


    # ----------------------------
    # SHORT TRADE
    # ----------------------------
    elif short_signal:
        print("Sell Signal: VWAP pullback continuation")

        entry_price = last_candle["bid_close"]
        stop_loss = df["ask_high"].iloc[-3:-1].max()

        risk = stop_loss - entry_price
        if risk <= 0:
            print("Invalid risk — skipping trade")
            return

        take_profit = entry_price - tp_ratio * risk
        reward = entry_price - take_profit

        place_order(
            direction="short",
            stop_loss=stop_loss,
            take_profit=take_profit
        )

        # ---- LOG TRADE ----
        trade_log.loc[len(trade_log)] = {
            "timestamp": last_candle.name,
            "instrument": instrument,
            "direction": "short",
            "entry_price": entry_price,
            "stop_loss": stop_loss,
            "take_profit": take_profit,
            "exit_price": None,
            "exit_reason": None,
            "pnl": None
        }


    else:
        print("Strategy conditions not met")


#### *Exit Detection & PnL Attribution*

Monitors OANDA transactions to detect when trades are closed
via stop loss or take profit and updates the trade log accordingly.


In [10]:
def update_trade_exits(client):
    global trade_log

    if trade_log.empty:
        return

    # Only look at trades without exits
    open_trades = trade_log[trade_log["exit_price"].isna()]

    if open_trades.empty:
        return

    r = transactions.TransactionsSinceID(
        config.OANDA_ACCOUNT_ID,
        transactionID=0
    )
    client.request(r)

    txns = r.response.get("transactions", [])

    for idx, trade in open_trades.iterrows():
        for t in txns:
            if t["type"] == "ORDER_FILL" and t.get("instrument") == trade["instrument"]:
                
                # Check if this is a closing fill
                if "tradeClosed" in t or "tradesClosed" in t:
                    exit_price = float(t["price"])
                    units = float(t["units"])

                    # Determine PnL
                    if trade["direction"] == "long":
                        pnl = (exit_price - trade["entry_price"]) * abs(units)
                    else:
                        pnl = (trade["entry_price"] - exit_price) * abs(units)

                    trade_log.loc[idx, "exit_price"] = exit_price
                    trade_log.loc[idx, "exit_reason"] = t.get("reason", "UNKNOWN")
                    trade_log.loc[idx, "pnl"] = pnl


#### *Bot Execution*

Runs one evaluation cycle:
1. Fetches latest market data
2. Computes indicators
3. Evaluates trade signals
4. Places orders if conditions are met
5. Updates trade exits


In [11]:
def run_bot():
    print("Starting trading bot")

    price = get_candles(instrument, timeframe, count, client)
    price = calculate_indicators(price)

    vwap_pullback_trade(price)

    # Check if any trades closed and log exits
    update_trade_exits(client)

In [12]:
run_bot()

Starting trading bot
Strategy conditions not met


#### *Trade Log Export*

Exports the trade log to CSV for offline analysis,
visualization, or performance evaluation.


In [13]:
trade_log.to_csv("trade_log.csv", index=False)
trade_log

,timestamp,instrument,direction,entry_price,stop_loss,take_profit,exit_price,exit_reason,pnl
